In [1]:
import os
import string
import glob
from tensorflow.keras.applications import MobileNet
import tensorflow.keras.applications.mobilenet  

from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow.keras.applications.inception_v3


from tqdm import tqdm
import tensorflow.keras.preprocessing.image
import pickle
from time import time
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers

from tensorflow.keras.models import Model

from tensorflow.keras.layers import add
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

START = "startseq"
STOP = "endseq"
EPOCHS = 10
USE_INCEPTION = True

In [3]:

    root_captioning = "C:\\Users\\hp\\Desktop\\Text Captions"
    root_captioning1 = "C:\\Users\\hp\\Desktop\\Flicker8k_Dataset"

In [4]:
null_punct = str.maketrans('', '', string.punctuation)
lookup = dict()

with open( os.path.join(root_captioning,'Flickr8k.token.txt'), 'r') as fp:
  
  max_length = 0
  for line in fp.read().split('\n'):
    tok = line.split()
    if len(line) >= 2:
      id = tok[0].split('.')[0]
      desc = tok[1:]
      
      # Cleanup description
      desc = [word.lower() for word in desc]
      desc = [w.translate(null_punct) for w in desc]
      desc = [word for word in desc if len(word)>1]
      desc = [word for word in desc if word.isalpha()]
      max_length = max(max_length,len(desc))
      
      if id not in lookup:
        lookup[id] = list()
      lookup[id].append(' '.join(desc))
      
lex = set()
for key in lookup:
  [lex.update(d.split()) for d in lookup[key]]


In [5]:
print(len(lookup)) # How many unique words
print(len(set(lex))) # The dictionary
print(max_length) # Maximum length of a caption (in words)


8092
8763
32


In [6]:
# Warning, running this too soon on GDrive can sometimes not work.
# Just rerun if len(img) = 0
import re
img = [f for f in os.listdir("C:\\Users\\hp\Desktop\\Flicker8k_Dataset") if re.match(r'[0-9]+.*\.jpg', f)]


In [10]:
len(img)

8091

In [7]:
train_images_path = os.path.join(root_captioning,'Flickr_8k.trainImages.txt') 
train_images = set(open(train_images_path, 'r').read().strip().split('\n'))
test_images_path = os.path.join(root_captioning,'Flickr_8k.testImages.txt') 
test_images = set(open(test_images_path, 'r').read().strip().split('\n'))

train_img = []
test_img = []

for i in img:
  f = os.path.split(i)[-1]
  if f in train_images: 
    train_img.append(f) 
  elif f in test_images:
    test_img.append(f) 

In [13]:
print(len(train_images))
print(len(test_images))

6000
1000


In [8]:
train_descriptions = {k:v for k,v in lookup.items() if f'{k}.jpg' in train_images}


In [9]:
train_descriptions

{'1000268201_693b08cb0e': ['child in pink dress is climbing up set of stairs in an entry way',
  'girl going into wooden building',
  'little girl climbing into wooden playhouse',
  'little girl climbing the stairs to her playhouse',
  'little girl in pink dress going into wooden cabin'],
 '1001773457_577c3a7d70': ['black dog and spotted dog are fighting',
  'black dog and tricolored dog playing with each other on the road',
  'black dog and white dog with brown spots are staring at each other in the street',
  'two dogs of different breeds looking at each other on the road',
  'two dogs on pavement moving toward each other'],
 '1002674143_1b742ab4b8': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl',
  'little girl is sitting in front of large painted rainbow',
  'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
  'there is girl with pigtails sitting in front of rainbow painting',
  'young girl with pi

In [16]:
for n,v in train_descriptions.items(): 
  for d in range(len(v)):
    v[d] = f'{START} {v[d]} {STOP}'

In [17]:
train_descriptions

{'1000268201_693b08cb0e': ['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
  'startseq girl going into wooden building endseq',
  'startseq little girl climbing into wooden playhouse endseq',
  'startseq little girl climbing the stairs to her playhouse endseq',
  'startseq little girl in pink dress going into wooden cabin endseq'],
 '1001773457_577c3a7d70': ['startseq black dog and spotted dog are fighting endseq',
  'startseq black dog and tricolored dog playing with each other on the road endseq',
  'startseq black dog and white dog with brown spots are staring at each other in the street endseq',
  'startseq two dogs of different breeds looking at each other on the road endseq',
  'startseq two dogs on pavement moving toward each other endseq'],
 '1002674143_1b742ab4b8': ['startseq little girl covered in paint sits in front of painted rainbow with her hands in bowl endseq',
  'startseq little girl is sitting in front of large painted rainbow endse

In [19]:
if USE_INCEPTION:
  encode_model = InceptionV3(weights='imagenet')
  encode_model = Model(encode_model.input, encode_model.layers[-2].output)
  WIDTH = 299
  HEIGHT = 299
  OUTPUT_DIM = 2048
  preprocess_input = tensorflow.keras.applications.inception_v3.preprocess_input


In [20]:
encode_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
______________________________________________________________________________________________

__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 35, 35, 64)   192         conv2d_113[0][0]                 
__________________________________________________________________________________________________
batch_normalization_115 (BatchN (None, 35, 35, 64)   192         conv2d_115[0][0]                 
__________________________________________________________________________________________________
batch_normalization_118 (BatchN (None, 35, 35, 96)   288         conv2d_118[0][0]                 
__________________________________________________________________________________________________
batch_normalization_119 (BatchN (None, 35, 35, 64)   192         conv2d_119[0][0]                 
__________________________________________________________________________________________________
activation_113 (Activation)     (None, 35, 35, 64)   0           batch_normalization_113[0][0]    
__________

__________________________________________________________________________________________________
batch_normalization_134 (BatchN (None, 17, 17, 192)  576         conv2d_134[0][0]                 
__________________________________________________________________________________________________
batch_normalization_137 (BatchN (None, 17, 17, 192)  576         conv2d_137[0][0]                 
__________________________________________________________________________________________________
batch_normalization_142 (BatchN (None, 17, 17, 192)  576         conv2d_142[0][0]                 
__________________________________________________________________________________________________
batch_normalization_143 (BatchN (None, 17, 17, 192)  576         conv2d_143[0][0]                 
__________________________________________________________________________________________________
activation_134 (Activation)     (None, 17, 17, 192)  0           batch_normalization_134[0][0]    
__________

batch_normalization_167 (BatchN (None, 17, 17, 192)  576         conv2d_167[0][0]                 
__________________________________________________________________________________________________
activation_167 (Activation)     (None, 17, 17, 192)  0           batch_normalization_167[0][0]    
__________________________________________________________________________________________________
conv2d_164 (Conv2D)             (None, 17, 17, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_168 (Conv2D)             (None, 17, 17, 192)  258048      activation_167[0][0]             
__________________________________________________________________________________________________
batch_normalization_164 (BatchN (None, 17, 17, 192)  576         conv2d_164[0][0]                 
__________________________________________________________________________________________________
batch_norm

                                                                 mixed9_1[0][0]                   
                                                                 concatenate_3[0][0]              
                                                                 activation_187[0][0]             
__________________________________________________________________________________________________
avg_pool (GlobalAveragePooling2 (None, 2048)         0           mixed10[0][0]                    
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
__________________________________________________________________________________________________


In [31]:
def encodeImage(img):
  img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
  x = tensorflow.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  x = encode_model.predict(x) # Get the encoding vector for the image
  x = np.reshape(x, OUTPUT_DIM )
  return x

In [37]:
train_path = os.path.join(root_captioning,f'train{OUTPUT_DIM}.pkl')
if not os.path.exists(train_path):
  start = time()
  encoding_train = {}
  for id in tqdm(train_img):
    image_path = os.path.join(root_captioning1, id)
    img = tensorflow.keras.preprocessing.image.load_img(image_path, target_size=(HEIGHT, WIDTH))
    encoding_train[id] = encodeImage(img)
  with open(train_path, "wb") as fp:
    pickle.dump(encoding_train, fp)
  print(f"\nGenerating training set took: {hms_string(time()-start)}")
else:
  with open(train_path, "rb") as fp:
    encoding_train = pickle.load(fp)

100%|██████████████████████████████████████████████████████████████████████████████| 6000/6000 [30:49<00:00,  3.24it/s]



Generating training set took: 0:30:49.76


In [39]:
test_path = os.path.join(root_captioning,f'test{OUTPUT_DIM}.pkl')
if not os.path.exists(test_path):
  start = time()
  encoding_test = {}
  for id in tqdm(test_img):
    image_path = os.path.join(root_captioning1, id)
    img = tensorflow.keras.preprocessing.image.load_img(image_path, target_size=(HEIGHT, WIDTH))
    encoding_test[id] = encodeImage(img)
  with open(test_path, "wb") as fp:
    pickle.dump(encoding_test, fp)
  print(f"\nGenerating testing set took: {hms_string(time()-start)}")
else:
  with open(test_path, "rb") as fp:
    encoding_test = pickle.load(fp)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:07<00:00,  2.72it/s]



Generating testing set took: 0:06:08.00


Next we separate the captions that will be usef for training.  There are two sides to this training, the images and the captions.

In [10]:
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
(all_train_captions)

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin',
 'black dog and spotted dog are fighting',
 'black dog and tricolored dog playing with each other on the road',
 'black dog and white dog with brown spots are staring at each other in the street',
 'two dogs of different breeds looking at each other on the road',
 'two dogs on pavement moving toward each other',
 'little girl covered in paint sits in front of painted rainbow with her hands in bowl',
 'little girl is sitting in front of large painted rainbow',
 'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
 'there is girl with pigtails sitting in front of rainbow painting',
 'young girl with pigtails painting outside in the grass',
 'man lays on bench while his dog sits by him',
 'man

Words that do not occur very often can be misleading to neural network training.  It is better to simply remove such words.  Here we remove any words that occur less than 10 times.  We display what the total vocabulary shrunk to.

In [11]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d ==> %d' % (len(word_counts), len(vocab)))

preprocessed words 7576 ==> 1649


In [12]:
vocab

['child',
 'in',
 'pink',
 'dress',
 'is',
 'climbing',
 'up',
 'set',
 'of',
 'stairs',
 'an',
 'way',
 'girl',
 'going',
 'into',
 'wooden',
 'building',
 'little',
 'the',
 'to',
 'her',
 'black',
 'dog',
 'and',
 'spotted',
 'are',
 'fighting',
 'tricolored',
 'playing',
 'with',
 'each',
 'other',
 'on',
 'road',
 'white',
 'brown',
 'spots',
 'staring',
 'at',
 'street',
 'two',
 'dogs',
 'different',
 'looking',
 'pavement',
 'moving',
 'toward',
 'covered',
 'paint',
 'sits',
 'front',
 'painted',
 'rainbow',
 'hands',
 'bowl',
 'sitting',
 'large',
 'small',
 'grass',
 'plays',
 'it',
 'there',
 'pigtails',
 'painting',
 'young',
 'outside',
 'man',
 'lays',
 'bench',
 'while',
 'his',
 'by',
 'him',
 'which',
 'also',
 'tied',
 'sleeping',
 'next',
 'shirtless',
 'lies',
 'park',
 'laying',
 'holding',
 'leash',
 'ground',
 'orange',
 'hat',
 'something',
 'wears',
 'glasses',
 'wearing',
 'beer',
 'can',
 'ears',
 'rope',
 'net',
 'red',
 'climbs',
 'bridge',
 'onto',
 'rope

Next we build two lookup tables for this vocabulary. One idxtoword convers index numbers to actual words to index values.  The wordtoidx lookup table performs the opposit.

In [13]:
idxtoword = {}
wordtoidx = {}

ix = 1
for w in vocab:
    wordtoidx[w] = ix
    idxtoword[ix] = w
    ix += 1
    
vocab_size = len(idxtoword) + 1 
vocab_size

1650

In [14]:
wordtoidx

{'child': 1,
 'in': 2,
 'pink': 3,
 'dress': 4,
 'is': 5,
 'climbing': 6,
 'up': 7,
 'set': 8,
 'of': 9,
 'stairs': 10,
 'an': 11,
 'way': 12,
 'girl': 13,
 'going': 14,
 'into': 15,
 'wooden': 16,
 'building': 17,
 'little': 18,
 'the': 19,
 'to': 20,
 'her': 21,
 'black': 22,
 'dog': 23,
 'and': 24,
 'spotted': 25,
 'are': 26,
 'fighting': 27,
 'tricolored': 28,
 'playing': 29,
 'with': 30,
 'each': 31,
 'other': 32,
 'on': 33,
 'road': 34,
 'white': 35,
 'brown': 36,
 'spots': 37,
 'staring': 38,
 'at': 39,
 'street': 40,
 'two': 41,
 'dogs': 42,
 'different': 43,
 'looking': 44,
 'pavement': 45,
 'moving': 46,
 'toward': 47,
 'covered': 48,
 'paint': 49,
 'sits': 50,
 'front': 51,
 'painted': 52,
 'rainbow': 53,
 'hands': 54,
 'bowl': 55,
 'sitting': 56,
 'large': 57,
 'small': 58,
 'grass': 59,
 'plays': 60,
 'it': 61,
 'there': 62,
 'pigtails': 63,
 'painting': 64,
 'young': 65,
 'outside': 66,
 'man': 67,
 'lays': 68,
 'bench': 69,
 'while': 70,
 'his': 71,
 'by': 72,
 'him': 73

Previously we added a start and stop token to all sentences.  We must account for this in the maximum length of captions.

In [15]:
max_length +=2
print(max_length)

34


In [117]:
def data_generator(descriptions, photos, wordtoidx, max_length, num_photos_per_batch):
  # x1 - Training data for photos
  # x2 - The caption that goes with each photo
  # y - The predicted rest of the caption
  x1, x2, y = [], [], []
  n=0
  while True:
    for key, desc_list in descriptions.items():
      n+=1
      photo = photos[key+'.jpg']
      # Each photo has 5 descriptions
      for desc in desc_list:
        # Convert each word into a list of sequences.
        seq = [wordtoidx[word] for word in desc.split(' ') if word in wordtoidx]
        # Generate a training case for every possible sequence and outcome
        for i in range(1, len(seq)):
          in_seq, out_seq = seq[:i], seq[i]
          in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
          out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
          x1.append(photo)
          x2.append(in_seq)
          y.append(out_seq)
      if n==num_photos_per_batch:
        yield ([np.array(x1), np.array(x2)], np.array(y))
        x1, x2, y = [], [], []
        n=0

In [116]:
y

[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)]

In [119]:
generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, 3)

In [92]:
# desc_list=train_descriptions['1000268201_693b08cb0e']
# for desc in desc_list:
#     # Convert each word into a list of sequences.
#     seq = [wordtoidx[word] for word in desc.split(' ') if word in wordtoidx]


In [98]:
x1, x2, y = [], [], []
photo = '1000268201_693b08cb0e.jpg'

for i in range(1,len(seq)):
    in_seq, out_seq = seq[:i], seq[i]
    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
    x1.append(photo)
    x2.append(in_seq)
    y.append(out_seq)

In [100]:
x1, x2, y,len(seq)

(['1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg',
  '1000268201_693b08cb0e.jpg'],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
  array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 20]),
  array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 20, 15]),
  array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 20, 15,  3]),
  array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [103]:
train_descriptions1={'1000268201_693b08cb0e': ['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
  'startseq girl going into wooden building endseq',
  'startseq little girl climbing into wooden playhouse endseq',
  'startseq little girl climbing the stairs to her playhouse endseq',
  'startseq little girl in pink dress going into wooden cabin endseq']}

In [104]:
train_descriptions1['1000268201_693b08cb0e']

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

### Loading Glove Embeddings

In [120]:
embeddings_index = {} 
f = open(os.path.join("C:\\Users\\hp\\Desktop\\Text Captions\\", 'glove.6B.200d.txt'), encoding="utf-8")

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print(f'Found {len(embeddings_index)} word vectors.')

400000it [01:03, 6332.82it/s]


Found 400000 word vectors.


In [123]:
embedding_dim = 200

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoidx.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [124]:
embedding_matrix.shape

(1652, 200)

In [125]:
inputs1 = Input(shape=(OUTPUT_DIM,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
caption_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [126]:
embedding_dim

200

In [127]:
caption_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 34, 200)      330400      input_4[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_3[0][0]                    
____________________________________________________________________________________________

In [128]:
caption_model.layers[2].set_weights([embedding_matrix])
caption_model.layers[2].trainable = False
caption_model.compile(loss='categorical_crossentropy', optimizer='adam')

### Train the Neural Network

In [129]:
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [131]:
model_path = os.path.join(root_captioning,f'caption-model.hdf5')
if not os.path.exists(model_path):
  for i in tqdm(range(EPOCHS*2)):
      generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath)
      caption_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

  caption_model.optimizer.lr = 1e-4
  number_pics_per_bath = 6
  steps = len(train_descriptions)//number_pics_per_bath

  for i in range(EPOCHS):
      generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_bath)
      caption_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)  
  caption_model.save_weights(model_path)
  print(f"\Training took: {hms_string(time()-start)}")
else:
  caption_model.load_weights(model_path)
    




  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

 314/2000 [===>..........................] - ETA: 36:05 - loss: 3.20 - ETA: 35:37 - loss: 3.28 - ETA: 34:39 - loss: 3.18 - ETA: 33:15 - loss: 3.07 - ETA: 33:58 - loss: 3.05 - ETA: 33:26 - loss: 3.04 - ETA: 32:54 - loss: 3.09 - ETA: 32:57 - loss: 3.05 - ETA: 32:44 - loss: 3.09 - ETA: 32:40 - loss: 3.13 - ETA: 32:38 - loss: 3.14 - ETA: 32:22 - loss: 3.16 - ETA: 32:13 - loss: 3.16 - ETA: 32:01 - loss: 3.16 - ETA: 31:51 - loss: 3.13 - ETA: 31:37 - loss: 3.14 - ETA: 31:33 - loss: 3.16 - ETA: 31:30 - loss: 3.19 - ETA: 31:19 - loss: 3.17 - ETA: 31:12 - loss: 3.20 - ETA: 31:03 - loss: 3.22 - ETA: 30:49 - loss: 3.21 - ETA: 30:40 - loss: 3.18 - ETA: 30:27 - loss: 3.18 - ETA: 30:18 - loss: 3.19 - ETA: 31:09 - loss: 3.20 - ETA: 31:06 - loss: 3.22 - ETA: 31:03 - loss: 3.21 - ETA: 31:00 - loss: 3.19 - ETA: 31:07 - loss: 3.18 - ETA: 31:12 - loss: 3.15 - ETA: 31:27 - loss: 3.17 - ETA: 31:25 - loss: 3.17 - ETA: 31:29 - loss: 3.17 - ETA: 31:24 - loss: 3.20 - ETA: 31:17 - loss: 3.19 - ETA: 31:11 - loss: 

 628/2000 [========>.....................] - ETA: 26:36 - loss: 3.52 - ETA: 26:35 - loss: 3.52 - ETA: 26:34 - loss: 3.52 - ETA: 26:33 - loss: 3.52 - ETA: 26:32 - loss: 3.52 - ETA: 26:31 - loss: 3.52 - ETA: 26:30 - loss: 3.52 - ETA: 26:29 - loss: 3.52 - ETA: 26:28 - loss: 3.52 - ETA: 26:27 - loss: 3.52 - ETA: 26:26 - loss: 3.52 - ETA: 26:25 - loss: 3.52 - ETA: 26:24 - loss: 3.52 - ETA: 26:22 - loss: 3.52 - ETA: 26:21 - loss: 3.52 - ETA: 26:20 - loss: 3.52 - ETA: 26:18 - loss: 3.52 - ETA: 26:18 - loss: 3.52 - ETA: 26:17 - loss: 3.52 - ETA: 26:16 - loss: 3.52 - ETA: 26:15 - loss: 3.52 - ETA: 26:13 - loss: 3.52 - ETA: 26:12 - loss: 3.52 - ETA: 26:10 - loss: 3.52 - ETA: 26:09 - loss: 3.52 - ETA: 26:08 - loss: 3.52 - ETA: 26:06 - loss: 3.52 - ETA: 26:05 - loss: 3.52 - ETA: 26:03 - loss: 3.52 - ETA: 26:02 - loss: 3.52 - ETA: 26:01 - loss: 3.52 - ETA: 25:59 - loss: 3.51 - ETA: 25:58 - loss: 3.51 - ETA: 25:57 - loss: 3.52 - ETA: 25:56 - loss: 3.51 - ETA: 25:55 - loss: 3.51 - ETA: 25:55 - loss: 

 942/2000 [=============>................] - ETA: 21:26 - loss: 3.46 - ETA: 21:26 - loss: 3.46 - ETA: 21:25 - loss: 3.46 - ETA: 21:24 - loss: 3.46 - ETA: 21:24 - loss: 3.46 - ETA: 21:24 - loss: 3.46 - ETA: 21:23 - loss: 3.46 - ETA: 21:23 - loss: 3.46 - ETA: 21:22 - loss: 3.46 - ETA: 21:21 - loss: 3.46 - ETA: 21:21 - loss: 3.46 - ETA: 21:21 - loss: 3.46 - ETA: 21:20 - loss: 3.46 - ETA: 21:20 - loss: 3.46 - ETA: 21:20 - loss: 3.46 - ETA: 21:20 - loss: 3.46 - ETA: 21:20 - loss: 3.46 - ETA: 21:19 - loss: 3.46 - ETA: 21:19 - loss: 3.46 - ETA: 21:18 - loss: 3.46 - ETA: 21:17 - loss: 3.46 - ETA: 21:17 - loss: 3.46 - ETA: 21:17 - loss: 3.46 - ETA: 21:17 - loss: 3.46 - ETA: 21:16 - loss: 3.46 - ETA: 21:15 - loss: 3.46 - ETA: 21:14 - loss: 3.46 - ETA: 21:13 - loss: 3.46 - ETA: 21:12 - loss: 3.46 - ETA: 21:12 - loss: 3.46 - ETA: 21:11 - loss: 3.46 - ETA: 21:10 - loss: 3.46 - ETA: 21:09 - loss: 3.46 - ETA: 21:08 - loss: 3.46 - ETA: 21:07 - loss: 3.46 - ETA: 21:06 - loss: 3.46 - ETA: 21:05 - loss: 

1256/2000 [=================>............] - ETA: 17:05 - loss: 3.44 - ETA: 17:04 - loss: 3.44 - ETA: 17:04 - loss: 3.44 - ETA: 17:03 - loss: 3.44 - ETA: 17:02 - loss: 3.44 - ETA: 17:01 - loss: 3.44 - ETA: 17:00 - loss: 3.44 - ETA: 16:59 - loss: 3.44 - ETA: 16:58 - loss: 3.44 - ETA: 16:58 - loss: 3.44 - ETA: 16:57 - loss: 3.45 - ETA: 16:56 - loss: 3.45 - ETA: 16:55 - loss: 3.45 - ETA: 16:55 - loss: 3.45 - ETA: 16:53 - loss: 3.45 - ETA: 16:53 - loss: 3.45 - ETA: 16:52 - loss: 3.45 - ETA: 16:51 - loss: 3.45 - ETA: 16:50 - loss: 3.45 - ETA: 16:49 - loss: 3.45 - ETA: 16:48 - loss: 3.45 - ETA: 16:47 - loss: 3.45 - ETA: 16:46 - loss: 3.45 - ETA: 16:45 - loss: 3.45 - ETA: 16:44 - loss: 3.45 - ETA: 16:43 - loss: 3.45 - ETA: 16:42 - loss: 3.44 - ETA: 16:41 - loss: 3.44 - ETA: 16:40 - loss: 3.44 - ETA: 16:40 - loss: 3.44 - ETA: 16:39 - loss: 3.44 - ETA: 16:38 - loss: 3.44 - ETA: 16:38 - loss: 3.44 - ETA: 16:37 - loss: 3.44 - ETA: 16:36 - loss: 3.44 - ETA: 16:35 - loss: 3.44 - ETA: 16:34 - loss: 

...

 942/1000 [===========================>..] - ETA: 8:02 - loss: 2.405 - ETA: 8:01 - loss: 2.404 - ETA: 8:00 - loss: 2.405 - ETA: 7:58 - loss: 2.404 - ETA: 7:57 - loss: 2.404 - ETA: 7:56 - loss: 2.404 - ETA: 7:55 - loss: 2.404 - ETA: 7:53 - loss: 2.404 - ETA: 7:52 - loss: 2.403 - ETA: 7:51 - loss: 2.403 - ETA: 7:49 - loss: 2.403 - ETA: 7:48 - loss: 2.403 - ETA: 7:47 - loss: 2.403 - ETA: 7:45 - loss: 2.403 - ETA: 7:44 - loss: 2.403 - ETA: 7:43 - loss: 2.403 - ETA: 7:41 - loss: 2.404 - ETA: 7:40 - loss: 2.403 - ETA: 7:39 - loss: 2.403 - ETA: 7:37 - loss: 2.403 - ETA: 7:36 - loss: 2.403 - ETA: 7:35 - loss: 2.403 - ETA: 7:33 - loss: 2.402 - ETA: 7:32 - loss: 2.402 - ETA: 7:31 - loss: 2.402 - ETA: 7:30 - loss: 2.402 - ETA: 7:28 - loss: 2.402 - ETA: 7:27 - loss: 2.402 - ETA: 7:26 - loss: 2.402 - ETA: 7:24 - loss: 2.401 - ETA: 7:23 - loss: 2.401 - ETA: 7:22 - loss: 2.401 - ETA: 7:20 - loss: 2.402 - ETA: 7:19 - loss: 2.402 - ETA: 7:18 - loss: 2.402 - ETA: 7:16 - loss: 2.402 - ETA: 7:15 - loss: 2

1000/1000 [==============================] - ETA: 1:13 - loss: 2.392 - ETA: 1:12 - loss: 2.392 - ETA: 1:11 - loss: 2.392 - ETA: 1:10 - loss: 2.392 - ETA: 1:08 - loss: 2.392 - ETA: 1:07 - loss: 2.392 - ETA: 1:06 - loss: 2.392 - ETA: 1:04 - loss: 2.392 - ETA: 1:03 - loss: 2.392 - ETA: 1:02 - loss: 2.392 - ETA: 1:00 - loss: 2.392 - ETA: 59s - loss: 2.392 - ETA: 58s - loss: 2.39 - ETA: 57s - loss: 2.39 - ETA: 55s - loss: 2.39 - ETA: 54s - loss: 2.39 - ETA: 53s - loss: 2.39 - ETA: 51s - loss: 2.39 - ETA: 50s - loss: 2.39 - ETA: 49s - loss: 2.39 - ETA: 47s - loss: 2.39 - ETA: 46s - loss: 2.39 - ETA: 45s - loss: 2.39 - ETA: 44s - loss: 2.39 - ETA: 42s - loss: 2.39 - ETA: 41s - loss: 2.39 - ETA: 40s - loss: 2.39 - ETA: 38s - loss: 2.39 - ETA: 37s - loss: 2.39 - ETA: 36s - loss: 2.39 - ETA: 35s - loss: 2.39 - ETA: 33s - loss: 2.39 - ETA: 32s - loss: 2.39 - ETA: 31s - loss: 2.39 - ETA: 29s - loss: 2.39 - ETA: 28s - loss: 2.39 - ETA: 27s - loss: 2.39 - ETA: 25s - loss: 2.39 - ETA: 24s - loss: 2.3

 314/1000 [========>.....................] - ETA: 23:46 - loss: 2.45 - ETA: 23:36 - loss: 2.33 - ETA: 23:10 - loss: 2.29 - ETA: 22:54 - loss: 2.26 - ETA: 22:59 - loss: 2.37 - ETA: 22:31 - loss: 2.35 - ETA: 22:08 - loss: 2.34 - ETA: 22:08 - loss: 2.31 - ETA: 22:06 - loss: 2.35 - ETA: 21:56 - loss: 2.34 - ETA: 21:46 - loss: 2.34 - ETA: 21:36 - loss: 2.31 - ETA: 21:27 - loss: 2.33 - ETA: 21:22 - loss: 2.32 - ETA: 21:12 - loss: 2.31 - ETA: 21:07 - loss: 2.31 - ETA: 21:12 - loss: 2.33 - ETA: 21:11 - loss: 2.34 - ETA: 21:05 - loss: 2.36 - ETA: 21:11 - loss: 2.36 - ETA: 21:09 - loss: 2.36 - ETA: 21:10 - loss: 2.35 - ETA: 21:10 - loss: 2.35 - ETA: 21:08 - loss: 2.35 - ETA: 21:05 - loss: 2.35 - ETA: 21:04 - loss: 2.35 - ETA: 21:02 - loss: 2.34 - ETA: 21:00 - loss: 2.34 - ETA: 21:00 - loss: 2.35 - ETA: 21:03 - loss: 2.36 - ETA: 21:00 - loss: 2.36 - ETA: 20:58 - loss: 2.37 - ETA: 21:00 - loss: 2.36 - ETA: 21:02 - loss: 2.37 - ETA: 21:01 - loss: 2.38 - ETA: 21:00 - loss: 2.40 - ETA: 21:01 - loss: 

 942/1000 [===========================>..] - ETA: 9:20 - loss: 2.397 - ETA: 9:19 - loss: 2.396 - ETA: 9:17 - loss: 2.396 - ETA: 9:16 - loss: 2.396 - ETA: 9:15 - loss: 2.396 - ETA: 9:13 - loss: 2.396 - ETA: 9:12 - loss: 2.396 - ETA: 9:10 - loss: 2.395 - ETA: 9:08 - loss: 2.395 - ETA: 9:07 - loss: 2.395 - ETA: 9:06 - loss: 2.394 - ETA: 9:04 - loss: 2.395 - ETA: 9:03 - loss: 2.394 - ETA: 9:01 - loss: 2.395 - ETA: 9:00 - loss: 2.395 - ETA: 8:58 - loss: 2.395 - ETA: 8:57 - loss: 2.395 - ETA: 8:55 - loss: 2.395 - ETA: 8:54 - loss: 2.394 - ETA: 8:52 - loss: 2.394 - ETA: 8:50 - loss: 2.394 - ETA: 8:49 - loss: 2.394 - ETA: 8:47 - loss: 2.393 - ETA: 8:46 - loss: 2.393 - ETA: 8:44 - loss: 2.394 - ETA: 8:43 - loss: 2.394 - ETA: 8:41 - loss: 2.394 - ETA: 8:40 - loss: 2.394 - ETA: 8:38 - loss: 2.394 - ETA: 8:36 - loss: 2.393 - ETA: 8:35 - loss: 2.393 - ETA: 8:34 - loss: 2.393 - ETA: 8:32 - loss: 2.393 - ETA: 8:31 - loss: 2.394 - ETA: 8:29 - loss: 2.394 - ETA: 8:28 - loss: 2.394 - ETA: 8:26 - loss: 2

1000/1000 [==============================] - ETA: 1:28 - loss: 2.385 - ETA: 1:27 - loss: 2.385 - ETA: 1:25 - loss: 2.385 - ETA: 1:24 - loss: 2.385 - ETA: 1:22 - loss: 2.385 - ETA: 1:21 - loss: 2.385 - ETA: 1:19 - loss: 2.385 - ETA: 1:18 - loss: 2.385 - ETA: 1:16 - loss: 2.385 - ETA: 1:14 - loss: 2.385 - ETA: 1:13 - loss: 2.385 - ETA: 1:11 - loss: 2.385 - ETA: 1:10 - loss: 2.385 - ETA: 1:08 - loss: 2.385 - ETA: 1:07 - loss: 2.385 - ETA: 1:05 - loss: 2.385 - ETA: 1:03 - loss: 2.384 - ETA: 1:02 - loss: 2.384 - ETA: 1:00 - loss: 2.384 - ETA: 59s - loss: 2.384 - ETA: 57s - loss: 2.38 - ETA: 56s - loss: 2.38 - ETA: 54s - loss: 2.38 - ETA: 52s - loss: 2.38 - ETA: 51s - loss: 2.38 - ETA: 49s - loss: 2.38 - ETA: 48s - loss: 2.38 - ETA: 46s - loss: 2.38 - ETA: 45s - loss: 2.38 - ETA: 43s - loss: 2.38 - ETA: 42s - loss: 2.38 - ETA: 40s - loss: 2.38 - ETA: 38s - loss: 2.38 - ETA: 37s - loss: 2.38 - ETA: 35s - loss: 2.38 - ETA: 34s - loss: 2.38 - ETA: 32s - loss: 2.38 - ETA: 31s - loss: 2.38 - ETA:

In [16]:
def generateCaption(photo):
    in_text = START
    for i in range(max_length):
        sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = caption_model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = idxtoword[yhat]
        in_text += ' ' + word
        if word == STOP:
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [19]:
for i in range(max_length):
    in_text = START
    sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]

In [20]:
sequence

[]

In [17]:
for z in range(10):
  pic = list(encoding_test.keys())[z]
  image = encoding_test[pic].reshape((1,OUTPUT_DIM))
  print(os.path.join(root_captioning1, pic))
  x=plt.imread(os.path.join(root_captioning1, pic))
  plt.imshow(x)
  plt.show()
  print("Caption:",generateCaption(image))
  print("_____________________________________")

NameError: name 'encoding_test' is not defined

In [137]:
encoding_test[pic].shape

(2048,)

In [ ]:
from PIL import Image, ImageFile
from matplotlib.pyplot import imshow
import requests
from io import BytesIO
import numpy as np

%matplotlib inline

urls = [
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/2015-03-09-phd-2nd-cluster-visit-1.png?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/about-jeff-heaton-2018.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/annie_dog.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/landscape.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/family_christmas.png?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/github_rock.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory.jpeg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory_coat.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory_exercise.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory_home.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory_n_stuffed.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory_run.jpeg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/hickory_sleep.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/jeff_books.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/jeff_cook.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/jeff_cube.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/jeff_laptop.jpg?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/jeff_laptops.png?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/road.JPG?raw=true",
  "https://github.com/jeffheaton/t81_558_deep_learning/blob/master/photos/snow_shovel.jpg?raw=true"
]

for url in urls:
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  img.load()

  plt.imshow(img)
  plt.show()
  
  response = requests.get(url)

  img = encodeImage(img).reshape((1,OUTPUT_DIM))
  print(img.shape)
  print("Caption:",generateCaption(img))
  print("_____________________________________")

In [146]:
a=Image.open("C:\\Users\\hp\\Pictures\\IMG-20200219-WA0047.jpg")
img = encodeImage(a).reshape((1,OUTPUT_DIM))
print(img.shape)
print("Caption:",generateCaption(img))
print("_____________________________________")

(1, 2048)
Caption: woman in black tank top is smiling
_____________________________________
